# pytorch 

|　フレームワーク | 主な特徴 |
|------------| ----------- |
|TensorFlow, Keras | 産業向け |
| PyTorch, Chainer | 研究向け（カスタマイズ性が高い） | 

## PyTorchの構成

| 構成内容 | 説明 |
|---------|-----|
|torch | メインのネームスペースでTensorや様々な数学関数がこのパッケージに含まれる。NumPyの構造を模している|
| torch.autograd | 自動微分のための関数が含まれる。自動微分のon/offを制御するコンテキストマネージャのenable_grad/no_gradや独自の微分可能関数を定義する際に使用する基底クラスであるFunctionなどが含まれる | 
| torch.nn | ニューラルネットワークを構築するための様々なデータ構造やレイヤーが定義されている。例えばConvolutionやLSTM、ReLUなどの活性化関数やMSELossなどの損失関数も含まれる |
| torch.optim | 確率的勾配降下（SGD）を中心としたパラメータ最適化アルゴリズムが実装されている | 
| torch.utils.data | SGDの繰り返し計算を回す際のミニバッチを作るためのユーティリティ関数が含まれている |
| torch.onnx | 	ONNX(Open Neural Network Exchange)1の形式でモデルをエクスポートするために使用する。 | 

## データの操作
PyTorch には、データを操作するための 2 つのプリミティブがあります: torch.utils.data.DataLoaderとtorch.utils.data.Dataset. Datasetサンプルとそれに対応するラベルを格納しDataLoader、 iterable で をラップしDatasetを作成します。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch は、 TorchText、 TorchVision、TorchAudioなどのドメイン固有のライブラリを提供し、そのすべてにデータセットが含まれています。
今回はTorchVision データセットを使用します。

このtorchvision.datasetsモジュールにはDataset、CIFAR、COCO などの多くの実世界のビジョン データのオブジェクトが含まれています。今回はFashionMNIST データセットを使用します。
すべての TorchVisionDatasetには 2 つの引数が含まれています

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Datasetを引数として DataLoaderに渡します。
これは、データセットを iterable でラップし、自動バッチ処理、サンプリング、シャッフル、マルチプロセス データ ロードをサポートします。ここでは、64 のバッチ サイズを定義します。
つまり、データローダー iterable の各要素は、64 個の機能とラベルのバッチを返します。

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break